# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
#Create a spark session:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Assignment').getOrCreate()
spark

In [4]:
#Import csv file:
df_cruise = spark.read.csv("cruise_ship_info.csv",header=True,inferSchema=True)
df_cruise.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [71]:
#To check the data types
df_cruise.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [72]:
#Five point summary:
df_cruise.describe().show()

+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|summary|Ship_name|Cruise_line|               Age|           Tonnage|       passengers|           length|            cabins|passenger_density|             crew|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|  count|      158|        158|               158|               158|              158|              158|               158|              158|              158|
|   mean| Infinity|       null|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|
| stddev|     null|       null| 7.615691058751413|37.229540025907866|9.677094775143416|1.793473548054825|4.4714172221480615| 8.63921711391542|3.503486564627034|
|    min|Adventure|    Azamara|   

In [13]:
#To import the vector assembler:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler 

In [14]:
#To print all the columns:

df_cruise.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [15]:
assembler = VectorAssembler(inputCols=['Age','Tonnage','passengers','length','cabins','passenger_density'],
                           outputCol='features')

In [19]:
data = assembler.transform(df_cruise)
data.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+--------------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|            features|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+--------------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|[6.0,30.276999999...|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|[6.0,30.276999999...|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|[26.0,47.262,14.8...|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|[11.0,110.0,29.74...|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|[17.0,101.353,26....|
|    Ecstasy|   Carnival| 22|            70.367|     20.

In [73]:
#Now we will drop all the columns except features and dependent feature
final_data = data.select('features','crew')
final_data.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [25]:
#Train test split 
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [30]:
#To check the number of observations in final data set:
final_data.count()

158

In [26]:
#To check the number of observations in training data:
train_data.count()

104

In [29]:
#To check the number of observations in test data:
test_data.count()

54

In [31]:
#To check five point summary point for the training data:
train_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              104|
|   mean|8.014807692307704|
| stddev|3.520072809476804|
|    min|              0.6|
|    max|             21.0|
+-------+-----------------+



In [32]:
#To check five point summary point for the test data:
test_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|               54|
|   mean|7.369259259259259|
| stddev|3.464121142513205|
|    min|             0.59|
|    max|             13.6|
+-------+-----------------+



In [34]:
from pyspark.ml.regression import LinearRegression

In [37]:
#Create a Linear Regression Model object
lr = LinearRegression(labelCol='crew')

In [42]:
# Fit the model to the data and call this model lrModel
lrmodel = lr.fit(train_data)

In [43]:
#To print the coefficients
lrmodel.coefficients

DenseVector([-0.0075, 0.0082, -0.1247, 0.4478, 0.7893, 0.0068])

In [45]:
#To print the intercept:
lrmodel.intercept

-1.1790050598400599

In [49]:
test_results = lrmodel.evaluate(test_data)

In [50]:
#To print the residuals:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| 0.19049875899233637|
| 0.14898199944648383|
| -0.3603298838882347|
|-0.34987648036405794|
| -0.5074979434803968|
| -0.5975150521276369|
| 0.49416544236714444|
| -0.4898156809924412|
|  0.9392318140874671|
| 0.09759419678714565|
| -0.6232984299780515|
| -0.4822807908029354|
|  0.2155737415289103|
|-0.46228434179944067|
|  0.8851053671410618|
| -0.4483560917744782|
|   0.804096551185653|
|-0.27226339386296683|
|  -1.272495465450211|
|  -0.508228649599042|
+--------------------+
only showing top 20 rows



C:\Users\User\anaconda3\lib\site-packages\pyspark\sql\context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [54]:
#To check the r2:
test_results.r2

0.9595570625962239

In [53]:
#To check the adjusted r2:
test_results.r2adj

0.9543941344170184

In [69]:
prediction = lrmodel.transform(test_data.select('features'))
prediction.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[5.0,122.0,28.5,1...| 6.509501241007664|
|[5.0,133.5,39.59,...|12.981018000553517|
|[6.0,30.276999999...|3.9103298838882345|
|[6.0,112.0,38.0,9...|11.249876480364058|
|[6.0,158.0,43.7,1...|14.107497943480396|
|[8.0,110.0,29.74,...|12.197515052127637|
|[9.0,88.5,21.24,9...| 9.805834557632856|
|[9.0,105.0,27.2,8...|11.169815680992441|
|[9.0,113.0,26.74,...|11.440768185912534|
|[10.0,68.0,10.8,7...| 6.262405803212855|
|[10.0,90.09,25.01...| 9.203298429978052|
|[10.0,105.0,27.2,...|11.162280790802935|
|[11.0,58.6,15.66,...| 7.384426258471089|
|[11.0,86.0,21.24,...| 9.762284341799441|
|[11.0,90.0,22.4,9...|10.114894632858938|
|[11.0,91.62700000...| 9.448356091774478|
|[11.0,108.977,26....|11.195903448814347|
|[12.0,50.0,7.0,7....| 4.722263393862967|
|[12.0,88.5,21.24,...|10.572495465450212|
|[12.0,90.09,25.01...| 9.188228649599042|
+--------------------+------------

In [70]:
print("RMSE: ", test_results.rootMeanSquaredError)
print("MSE: ", test_results.meanSquaredError)

RMSE:  0.6901690369923834
MSE:  0.47633329962299387
